## Crosswalk Model Annotation Tool

### Imports

In [1]:
import glob, os
import time
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import numpy as np
from PIL import Image
import matplotlib as mpl
import cv2

mpl.use('Agg')
import matplotlib.pyplot as plt

### Setup Config

In [2]:
# Image Locations
image_directory = './test_set'
# image_directory = '/Users/john/Documents/brown/csci1470/final-project/training/workspace/crosswalks/images/test'


# Save Annotations
save_directory = './test_annotated'
# save_directory = '/Users/john/Documents/brown/csci1470/final-project/training/workspace/crosswalks/images/test_annotated/'

# Labels
label_path = '../training/workspace/crosswalks/annotations/label_map.pbtxt'
# label_path = '/Users/john/Documents/brown/csci1470/final-project/training/workspace/crosswalks/annotations/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(label_path, use_display_name=True)

# Image Resize Function
def resize_image(image):
    return cv2.resize(image, dsize=(640, 640), interpolation=cv2.INTER_CUBIC)

# Load Image Function
def load_image_into_numpy_array(path):
    image_np = np.array(Image.open(path))

    if image_np.shape != (640, 640):
        image_np = resize_image(image_np)
        
    return image_np

### Load the Model

In [3]:
# Model Location
saved_model = '../training/workspace/crosswalks/exported-models/ssd_mobilenet_v2_fpnlite_640x640_7' + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(saved_model)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...

2022-12-10 14:58:43.288462: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Done! Took 7.705033779144287 seconds


### Run the Annotations

In [5]:
# Load images
image_paths = glob.glob(os.path.join(image_directory, '*.jpg'))

for image_path in image_paths:
    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {
        key: value[0, :num_detections].numpy()
        for key, value in detections.items()
    }
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(
        np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.20,
        agnostic_mode=False)

    plt.figure(figsize=(10, 10))
    plt.imshow(image_np_with_detections)

    (filepath, filename) = os.path.split(image_path)
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    save_path = os.path.join(save_directory, filename)
    plt.savefig(save_path)

Running inference for ./crosswalk-test-set/19_03_57_778197.jpg... Running inference for ./crosswalk-test-set/download.jpg... Running inference for ./crosswalk-test-set/test5.jpg... Running inference for ./crosswalk-test-set/test4.jpg... Running inference for ./crosswalk-test-set/test1.jpg... Running inference for ./crosswalk-test-set/test3.jpg... Running inference for ./crosswalk-test-set/test2.jpg... Running inference for ./crosswalk-test-set/test.jpg... Running inference for ./crosswalk-test-set/cross.jpg... 